In [83]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

#Global var
init_cap = 2000
price_limit = 10000 # don't open position if price exceeded price_limit
risk_free_rate = 5.0

#directory path
dir_path =r'D:\OneDrive\stock\Leleka\BuyBeforeEarnings\inputForPython'
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

if not csv_files:
    print('No CSV file in directory')



### Working with stock list filter dataframe

In [84]:

# add stock list to filter data
df_stock = pd.read_excel('InputStocks.xlsx')
# Using the str.cat() method without the underscore separator
df_stock['Y_Symbol'] = df_stock['Year'].astype(str).str.cat(df_stock['Symbol'], sep='')
df_stock['Y_Symbol'] = df_stock['Y_Symbol'].str.replace(' ', '')

In [85]:

#df_stock.to_excel('df_stock_result.xlsx')


### Working with CML download files

In [86]:
# Add all files in folder in one file
dataframes={}
all_data = []  # List to store all individual dataframes

for file in csv_files:
    file_path = os.path.join(dir_path, file)
    df = pd.read_csv(file_path)
    key_name = os.path.splitext(file)[0]
    dataframes[key_name] = df
    all_data.append(df)

# Concatenate all individual dataframes to form a single dataframe
combined_df = pd.concat(all_data, ignore_index=True)

In [87]:
#
# combined_df

In [88]:
# remove dollar symbol
combined_df['Profit/Loss'] = combined_df['Profit/Loss'].str.replace('$','', regex = False)
combined_df['Trade Price'] = combined_df['Trade Price'].str.replace('$','', regex = False)
combined_df['Stock Price'] = combined_df['Stock Price'].str.replace('$','', regex = False)
combined_df['Adjusted Stock Price'] = combined_df['Adjusted Stock Price'].str.replace('$','', regex = False)

# convert in numbers
combined_df['Profit/Loss']= pd.to_numeric(combined_df['Profit/Loss'],errors='coerce')
combined_df['Trade Price']= pd.to_numeric(combined_df['Trade Price'],errors='coerce')

# convert date 
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df = combined_df.reset_index(drop=True)


In [89]:
#combined_df.to_excel('combined_df.xlsx')


### Filter stocks

In [90]:
# Add year column and Y_Symbol column
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Y_Symbol'] = combined_df['Year'].astype(str).str.cat(combined_df['Symbol'], sep='')

# select only stocks from file 'InputStocks.xlsx'
combined_df = combined_df[combined_df['Y_Symbol'].isin(df_stock['Y_Symbol'])]

# sort by date to build chart
combined_df = combined_df.sort_values(by='Date')

### Calculate call spread price

In [91]:
# calculate CS open price
combined_df.loc[combined_df["Size"] == -1, "Trade Price"] *= -1  # if size -1 then we sold options it's reduce our calls costs
grouped_by_date = combined_df.groupby(['Date','Symbol'])[['Trade Price', 'Profit/Loss']].sum().reset_index()


In [92]:
# sorf dataframe by symbol and date
sorted_df = grouped_by_date.sort_values(by=['Symbol','Date'])


In [93]:
# delete result values of option // clean df
sorted_df.loc[sorted_df['Profit/Loss']!=0, 'Trade Price']=0


In [94]:
#sorted_df.to_excel('sorted.xlsx')

### Call spread price filter

In [95]:
# Here I delete rows where price_limit exceed my limit


# Identify rows where "Trade Price" is greater than price_limit
to_drop = sorted_df[sorted_df['Trade Price'] > price_limit]

# For each of these rows, identify the next row with the same "Symbol" and a subsequent date
drop_indices = []
for idx, row in to_drop.iterrows():
    drop_indices.append(idx)
    next_row = sorted_df[(sorted_df['Symbol'] == row['Symbol']) & (sorted_df['Date'] > row['Date'])].head(1)
    if not next_row.empty:
        drop_indices.append(next_row.index[0])

# Drop these rows from the DataFrame
sorted_df = sorted_df.drop(drop_indices)


In [96]:
# sorf dataframe by date
sorted_df = sorted_df.sort_values(by='Date')

### Working with cummulative columns

In [97]:
# making cummulative sum of profit
df_chart = sorted_df.copy()
df_chart['CumSum'] = df_chart['Profit/Loss'].cumsum()

# add SPY price to compare

start_date = df_chart['Date'].min()
end_date = df_chart['Date'].max()

# Fetch SPY data for the given date range
spy_data = yf.download('SPY', start=start_date, end=end_date)

# Merge the data
df_chart = df_chart.merge(spy_data[['Close']], left_on='Date', right_index=True, how='left')
df_chart.rename(columns={'Close': 'SPY Price'}, inplace=True)

[*********************100%***********************]  1 of 1 completed


In [98]:
df_chart

,Date,Symbol,Trade Price,Profit/Loss,CumSum,SPY Price
1,2007-01-03,GE,0.20,0.0,0.0,141.369995
0,2007-01-03,C,0.30,0.0,0.0,141.369995
2,2007-01-08,BAC,0.40,0.0,0.0,141.190002
3,2007-01-09,MSFT,0.70,0.0,0.0,141.070007
4,2007-01-09,T,0.45,0.0,0.0,141.070007
...,...,...,...,...,...,...
1061,2023-07-20,TSLA,0.00,-321.0,18979.0,452.179993
1062,2023-07-26,MSFT,0.00,3.0,18982.0,455.510010
1063,2023-07-26,V,0.00,-82.0,18900.0,455.510010
1064,2023-07-31,XOM,0.00,-24.0,18876.0,457.790009


In [99]:
df_chart.to_excel('df_chart.xlsx')

In [100]:
# add Year
df_chart['Year']=df_chart['Date'].dt.year

In [101]:
df_chart

,Date,Symbol,Trade Price,Profit/Loss,CumSum,SPY Price,Year
1,2007-01-03,GE,0.20,0.0,0.0,141.369995,2007
0,2007-01-03,C,0.30,0.0,0.0,141.369995,2007
2,2007-01-08,BAC,0.40,0.0,0.0,141.190002,2007
3,2007-01-09,MSFT,0.70,0.0,0.0,141.070007,2007
4,2007-01-09,T,0.45,0.0,0.0,141.070007,2007
...,...,...,...,...,...,...,...
1061,2023-07-20,TSLA,0.00,-321.0,18979.0,452.179993,2023
1062,2023-07-26,MSFT,0.00,3.0,18982.0,455.510010,2023
1063,2023-07-26,V,0.00,-82.0,18900.0,455.510010,2023
1064,2023-07-31,XOM,0.00,-24.0,18876.0,457.790009,2023


In [102]:
# Load the provided dataframe

# Plot using plotly.express
fig = px.line(df_chart, x='Date', y='CumSum', title='Cumulative Sum over Date')

#fig.add_trace(go.Scatter(x=df_chart_comp['Date'], y=df_chart_comp['SPY_buy_hold'], mode='lines', name='SPY_buy_hold'))

fig.show()



## Metrics

### Average win

In [103]:
# Calculate average win and average loss in absolute values for filtered data
average_win_filtered = df_chart[df_chart["Profit/Loss"] > 0]["Profit/Loss"].mean()
average_loss_filtered = abs(df_chart[df_chart["Profit/Loss"] < 0]["Profit/Loss"].mean())

average_win_filtered, average_loss_filtered




(189.20912547528516, 116.19101123595506)

In [104]:
# Filter out rows with "Profit/Loss" values of 0
filtered_df = df_chart[df_chart["Profit/Loss"] != 0]

# Calculate percentage of win trades and percentage of loss trades without considering 0s
total_trades_filtered = len(filtered_df)
win_trades_filtered = len(filtered_df[filtered_df["Profit/Loss"] > 0])
loss_trades_filtered = len(filtered_df[filtered_df["Profit/Loss"] < 0])

percent_win_trades_filtered = (win_trades_filtered / total_trades_filtered) * 100
percent_loss_trades_filtered = (loss_trades_filtered / total_trades_filtered) * 100

percent_win_trades_filtered, percent_loss_trades_filtered


(49.62264150943396, 50.37735849056604)

In [105]:
# Recalculate the mathematical expectation for filtered data
prob_win_filtered = percent_win_trades_filtered / 100
prob_loss_filtered = percent_loss_trades_filtered / 100

math_expectation_filtered = (prob_win_filtered * average_win_filtered) - (prob_loss_filtered * average_loss_filtered)

math_expectation_filtered


35.35660377358491

In [106]:


# Extract the year from the 'Date' column and group by it to calculate the annual profit
annual_profit = df_chart.groupby(df_chart['Year'])['Profit/Loss'].sum().reset_index()
annual_profit.columns = ['Year', 'Annual Profit']

annual_profit


,Year,Annual Profit
0,2007,-444.0
1,2008,-1691.0
2,2009,934.0
3,2010,-116.0
4,2011,2549.0
5,2012,-3170.0
6,2013,650.0
7,2014,232.0
8,2015,2384.0
9,2016,773.0


In [107]:
# ### Annualized return
# total_return = df_chart_comp['Profit/Loss'].sum()
# # Calculate the total number of days the strategy ran
# num_days = (df_chart_comp['Date'].iloc[-1] - df_chart_comp['Date'].iloc[0]).days

# # Calculate the annualized return using the total return and number of days
# annualized_return = ((1 + total_return / df_chart_comp['CumSum'].iloc[0]) ** (365.0 / num_days)) - 1

# annualized_return = round(annualized_return*100,2)
# annualized_return

In [108]:
# ### Annualized return SPY
# # Calculate the total number of days the data covers (for the cleaned data)
# num_days_spy = (df_chart_comp_cleaned['Date'].iloc[-1] - df_chart_comp_cleaned['Date'].iloc[0]).days

# # Calculate the annualized return for SPY using the total return and number of days
# spy_annualized_return = ((1 + spy_total_return_cleaned) ** (365.0 / num_days_spy)) - 1

# spy_annualized_return =round(spy_annualized_return*100,2)
# spy_annualized_return


### Volatility (Standard Deviation):

In [109]:
# # 1. Compute the daily returns
# df_chart_comp['Daily Returns'] = df_chart_comp['CumSum'].pct_change()

# # 2. Calculate the standard deviation of these daily returns
# daily_volatility = df_chart_comp['Daily Returns'].std()

# # 3. Annualize the standard deviation
# annualized_volatility = round(daily_volatility * (252**0.5),2)  # Using 252 trading days in a year

# annualized_volatility


### Drawdown

In [110]:
# # 1. Create a new column for cumulative balance
# df_chart_comp['Cumulative Balance'] = init_cap + df_chart_comp['Profit/Loss'].cumsum()

# # 2. Calculate running max for the cumulative balance
# running_max_balance = df_chart_comp['Cumulative Balance'].cummax()

# # 3. Calculate drawdowns as the decline from the running max balance
# drawdowns_balance = (df_chart_comp['Cumulative Balance'] - running_max_balance) / running_max_balance

# # 4. Identify the maximum drawdown
# max_drawdown_balance = drawdowns_balance.min()

# max_drawdown_balance


### consecutive loss trades
 

In [111]:
# 1. Create a binary column for loss days
df_loss_count = df_chart.copy()
df_loss_count['Loss Day'] = (df_loss_count['Profit/Loss'] < 0).astype(int)

# 2. Calculate the cumulative sum of loss days, but reset to zero whenever a non-loss day is encountered
df_loss_count['Consecutive Loss Days'] = df_loss_count['Loss Day'].groupby((df_loss_count['Loss Day'] == 0).cumsum()).cumsum()

# 3. Find the maximum number of consecutive loss days
max_consecutive_loss_days = df_loss_count['Consecutive Loss Days'].max()

max_consecutive_loss_days


9

In [112]:
#df_chart_comp_cleaned

## sharpe ratio

In [113]:
#average_annual = df_annual['Annual Profit'].mean()

## Sortino Ratio